# Vehicle Type Classification

## 1. Data Set

TODO: dataset information<br>
show data images

## 2. Data Preprocessing

In [1]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
import scipy.io as scio
import pandas as pd

In [2]:
mat = scio.loadmat('VehicleInfo.mat')
def alter(mat,path,object):
    info = mat.get('VehicleInfo')
    s = os.listdir(path)
    count = 1
    for i in sorted(s):
        base = os.path.basename(i)
        fileName = os.path.splitext(base)[0]
        result = []
        document = os.path.join(path,i)
        img = cv2.imread(document)
        if img is not None:
            left = info[count-2][0][3][0][0][0][0][0]
            top = info[count-2][0][3][0][0][1][0][0]
            right = info[count-2][0][3][0][0][2][0][0]
            bottom = info[count-2][0][3][0][0][3][0][0]
            img = img[top:bottom,left:right] 
            img = cv2.resize(img,(299,299))
        elif img is None:
            print('title')
        cv2.imwrite(object+os.sep+'%s.jpg' % fileName, img)
        count = count + 1

In [3]:
alter(mat,'BITVehicle_Dataset','Dataset_Resize')

Load image data and label information.<br>
Convert images and labels into numpy array.<br>
Spilt the data for train and test randomly.<br>

In [4]:
X_list = []
for filename in sorted(os.listdir('./Dataset_Resize/')):
    img = image.load_img(Path('./Dataset_Resize/', filename)) # load image data
    X_list.append(image.img_to_array(img))
X = np.array(X_list) # convert image to nparray

df = pd.read_csv('./vehicles.csv', index_col=0) # load annotation file
df_dummy = pd.get_dummies(df['Vehicle_Type']) # dummy encoding
Y = np.array(df_dummy)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2) # splite data for train and test

## 3. Set Up Environment

TODO: Introduce how to build aws enviroment<br>
a machine with Keras, TensorFlow installed

## 4. Train a baseline CNN (from scratch)

<p>We try to train one small CNN with a few layers and few filters per layer on our data, as an initial baseline refering to Keras offical examples<sup>[1]</sup> which is a VGG-like convnet and we resize the image to 32x32 in order to train it on our own computer cpu.</p>
<p>A convnet takes an image expressed as an array of numbers, applies a series of operations to that array and, at the end, returns the probability that an object in the image belongs to a particular class of objects. </p>

TODO: Introduce each layer<br>
This shallow CNN has 4 convolutional layers and uses ReLU(rectified linear units) activation function. And it uses maxpooling layers to down samples and dropout layers to reduce overfitting by a layer from seeing twice the exact same pattern.
In the first block, in the convolutional layers, there are 3 filters and the kernal size is 3x3, 

In [ ]:
def get_model_1(input_shape):
    model = Sequential()
    # input: 32x32 images with 3 channels -> (32, 32, 3) tensors.
    # this applies 32 convolution filters of size 3x3 each.
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape, name='Conv1-1'))
    model.add(Activation('relu', name='ReLU1-1'))
    model.add(Conv2D(32, (3, 3), name='Conv1-2'))
    model.add(Activation('relu', name='ReLU1-2'))
    model.add(MaxPooling2D(pool_size=(3, 3), name='MaxPool1'))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(6))
    model.add(Activation('softmax'))
    
    opt = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

Callbacks Method in Keras:<br>
TODO: Introduce callback functions

In [ ]:
early_Stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    verbose=0, 
    mode='auto'
)
reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)
checkpointer = ModelCheckpoint(filepath="CNN.hdf5", verbose=1)
csv = CSVLogger('CNN.csv')

TODO: fit method

In [ ]:
model_1 = get_model_1((32, 32, 3))
model.fit(X_train, 
          Y_train,
          batch_size=256,
          epochs=20,
          validation_data=(X_test, Y_test),
          shuffle=True,
          callbacks=[early_Stopping,reduceLR,checkpointer,csv])

### Get train result -> [Baseline CNN Model on Keras](./Keras Build CNN.ipynb)

# Reference

[1] https://keras.io/getting-started/sequential-model-guide/#examples

####  The code in the document by Junhan Ma, Bo Han is licensed under the MIT License https://opensource.org/licenses/MIT